# ChatBot Schedule
**In this project we will learn how to create a chatbot that allows you to check you and your friends events.**

Using this chatbot a multitude of users will be able to sync their google calendars to this application and through the use of this chat bot will be able to see what events all of the users have on particular days. For example, if we have three users, first all three of those users will need to allow the chatbot access to their google calendar and after that by asking the chatbot, "What do we have on Tuesday?", the chatbot will respond with the events all users have on that particular date.

# Imports

In [1]:
import random
import json
import pickle
import string
import numpy as np
import nltk
import string
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
# Us to lemmatize the individual words
lemmatizer = WordNetLemmatizer()
# load the intents json file
intents = json.loads(open('intents-sch.json').read())
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Tokenize
The process of tokenization is when we have a sentence and we chop up that sentence and extract each word from that sentence individually.

Here we will iterate over the intents in our intents json file, in order to get the words from the patterns
we made and we want to associate each pattern with its specific tag.

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['Whats', 'up'], 'greeting'), (['Hey'], 'greeting'), (['greetings'], 'greeting'), (['Good', 'morning'], 'greeting'), (['Good', 'evening'], 'greeting'), (['Good', 'afternoon'], 'greeting'), (['Nice', 'to', 'meet', 'you'], 'greeting'), (['See', 'you', 'soon'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Good', 'night'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['Goodbye'], 'goodbye'), (['My', 'name', 'is'], 'name'), (['I', 'am'], 'name'), (['I', 'go', 'by'], 'name'), (['My', 'full', 'name', 'is'], 'name'), (['My', 'friends', 'call', 'me'], 'name'), (['You', 'can', 'call', 'me'], 'name'), (['What', 'do', 'i', 'have', 'on'], 'schedule'), (['Do', 'i', 'have', 'plans', 'on'], 'schedule'), (['Am', 'i', 'busy', 'on'], 'schedule')]


# Creating Lemmas
What are lemmas? When we refer to the lemma of a certain word, we are referring to the stem of that word in the context of a sentence. For example, if we have this sentence, "I am running", the stem of the word "running" in the context of that sentence would be "run".

In [4]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

['Am', 'Do', 'Good', 'Goodbye', 'Have', 'Hello', 'Hey', 'Hi', 'I', 'My', 'Nice', 'See', 'What', 'Whats', 'You', 'a', 'afternoon', 'am', 'busy', 'by', 'bye', 'call', 'can', 'day', 'do', 'evening', 'friend', 'full', 'go', 'greeting', 'have', 'i', 'is', 'later', 'me', 'meet', 'morning', 'name', 'night', 'on', 'plan', 'soon', 'to', 'up', 'you']


In [5]:
classes = sorted(set(classes))

Save the datasets, words and classes, we created into a pickle file.

In [6]:
pickle.dump(words, open('words.pkl', 'wb'))

In [7]:
pickle.dump(classes, open('classes.pkl', 'wb'))

# Creating a Neural Net
In this next part we will be creating a neural net and feeding the data from our intents json file into our neural net so that we can try to understand what the user might be saying to the chatbot. We will create a convolutional neural net with the use of "reLu" layers, which we trained on our knowledge base for 200 epochs. What are epochs? A singular epoch refer to one cycle through the full training dataset, so when we refer to 200 epochs, that means that we are cycling 200 times through the full training dataset. We are training this model by giving it multiple patterns and helping it recognize which pattern fits with which tag, so for example, if the user says "Hello", this would be a pattern in the "greeting" tag in our knowledge base, so the model should be able to recognize and let us know that the user is trying to greet us and based of that information we can decide how to reply to the user.

For this model, it isn't exactly going to understand what the words that its receiving mean, so we need to represent these words as numerical values, so here we will set a word to either 0 or 1 based on whether or not that particular word occurs in the pattern.

In [8]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    group = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            group.append(1)
        else:
            group.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([group, output_row])

# Neural Network Model

Here we will train our neural network model. We can see, once we run the program, that after a certain amount of cycles our accuracy slowly becomes stable.

In [9]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist)
print("done")

C:\Users\shanm\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Train on 24 samples
Epoch 1/200
24/24 [==============================] - 1s 21ms/sample - loss: 1.4477 - accuracy: 0.2083
Epoch 2/200
24/24 [==============================] - 0s 333us/sample - loss: 1.3705 - accuracy: 0.3333
Epoch 3/200
24/24 [==============================] - 0s 541us/sample - loss: 1.4015 - accuracy: 0.4583
Epoch 4/200
24/24 [==============================] - 0s 500us/sample - loss: 1.2478 - accuracy: 0.6667
Epoch 5/200
24/24 [==============================] - 0s 500us/sample - loss: 1.2975 - accuracy: 0.5417
Epoch 6/200
24/24 [==============================] - 0s 541us/sample - loss: 1.2419 - accuracy: 0.5417
Epoch 7/200
24/24 [==============================] - 0s 500us/sample - loss: 1.1478 - accuracy: 0.5833
Epoch 8/200
24/24 [==============================] - 0s 624us/sample - loss: 1.1844 - accuracy: 0.6250
Epoch 9/200
24/24 [==============================] - 0s 541us/sample - loss: 1.1546 - accuracy: 0.5833
Epoch 10/200
24/24 [==============================] - 

24/24 [==============================] - 0s 541us/sample - loss: 0.1611 - accuracy: 0.9167
Epoch 80/200
24/24 [==============================] - 0s 458us/sample - loss: 0.1803 - accuracy: 0.9583
Epoch 81/200
24/24 [==============================] - 0s 458us/sample - loss: 0.1155 - accuracy: 0.9583
Epoch 82/200
24/24 [==============================] - 0s 583us/sample - loss: 0.1678 - accuracy: 0.9167
Epoch 83/200
24/24 [==============================] - 0s 375us/sample - loss: 0.1855 - accuracy: 0.9583
Epoch 84/200
24/24 [==============================] - 0s 583us/sample - loss: 0.1489 - accuracy: 0.9583
Epoch 85/200
24/24 [==============================] - 0s 541us/sample - loss: 0.2203 - accuracy: 0.8750
Epoch 86/200
24/24 [==============================] - 0s 541us/sample - loss: 0.1171 - accuracy: 0.9583
Epoch 87/200
24/24 [==============================] - 0s 416us/sample - loss: 0.2534 - accuracy: 0.9583
Epoch 88/200
24/24 [==============================] - 0s 541us/sample - loss:

24/24 [==============================] - 0s 458us/sample - loss: 0.1257 - accuracy: 0.9583
Epoch 158/200
24/24 [==============================] - 0s 541us/sample - loss: 0.1639 - accuracy: 0.9583
Epoch 159/200
24/24 [==============================] - 0s 416us/sample - loss: 0.1070 - accuracy: 0.9583
Epoch 160/200
24/24 [==============================] - 0s 500us/sample - loss: 0.1242 - accuracy: 0.9583
Epoch 161/200
24/24 [==============================] - 0s 541us/sample - loss: 0.1622 - accuracy: 0.9583
Epoch 162/200
24/24 [==============================] - 0s 583us/sample - loss: 0.1033 - accuracy: 0.9583
Epoch 163/200
24/24 [==============================] - 0s 583us/sample - loss: 0.1462 - accuracy: 0.9167
Epoch 164/200
24/24 [==============================] - 0s 458us/sample - loss: 0.1096 - accuracy: 0.9583
Epoch 165/200
24/24 [==============================] - 0s 458us/sample - loss: 0.0972 - accuracy: 0.9583
Epoch 166/200
24/24 [==============================] - 0s 416us/sampl